In [1]:
import numpy as np
import pandas as pd
import nbimporter
from sklearn.preprocessing import MinMaxScaler
from scipy.sparse import csr_matrix
from NMF_Model import *


Active user's ratings for the selected games:
   Game                   Rating
1)        Worms Reloaded  4     
2)          Brtal Legend  4     
3)         Killing Floor  5     
4)  Kerbal Space Program  5     
5)              PAYDAY 2  5     

Evaluation Metrics for Different Recommendation Sizes:
 Top K     RMSE      MAE  Recall@K
     5 0.023963 0.005549  0.872697
    10 0.023963 0.005549  0.933066
    15 0.023963 0.005549  0.955234


In [ ]:
# Load or define the original df (user-item ratings matrix)
df = pd.read_csv('./data/combined_final_dataset.csv')  # Adjust this path to where your data is stored

# Create pivot table for user-item ratings matrix
pv_sparse, pv = create_pivot_table(df)

# Simulate some random games for the active user (this could come from user input)
random_games = np.random.choice(pv.columns, 5, replace=False)

# Simulate new user ratings for these random games (active user's ratings)
new_user_ratings = {
    'game_name': random_games.tolist(),  # Randomly selected games from the user-item matrix
    'rating': [4, 4, 5, 5, 5]  # Ratings given by the active user
}

# Print the selected games for the active user to rate in a table format
print("\nActive user's ratings for the selected games:")
new_user_df = pd.DataFrame(new_user_ratings)
new_user_df.index = [f"{i + 1})" for i in range(len(new_user_df))]
new_user_df.columns = ["Game", "Rating"]
print(new_user_df.to_string(index=True, col_space=0, justify='left', header=True))

# Add the new user's ratings to the user-item matrix (pv)
new_user_id = 'new_user'  # We will use a unique ID for the new user
new_user_df['user_id'] = new_user_id

# Append the new ratings to the existing dataset
df_with_new_user = pd.concat([df, new_user_df[['user_id', 'Game', 'Rating']].rename(columns={"Game": "game_name", 
                                                                                             "Rating": "rating"})])

# Recreate the user-item matrix including the new user
pv_with_new_user = df_with_new_user.pivot_table(index=['user_id'], columns=['game_name'], values='rating')

# Normalize the ratings to ensure non-negative values for NMF
scaler = MinMaxScaler()  # Using MinMaxScaler to scale values between 0 and 1
pv_scaled_with_new_user = pd.DataFrame(scaler.fit_transform(pv_with_new_user.fillna(0)), 
                                       columns=pv_with_new_user.columns, index=pv_with_new_user.index)

# Fill missing values with 0 (indicating no rating)
pv_scaled_with_new_user = pv_scaled_with_new_user.fillna(0)

# Convert the pivot table to a sparse matrix format (CSR format)
pv_sparse_with_new_user = csr_matrix(pv_scaled_with_new_user.values)

# Evaluate NMF model for different recommendation sizes
results = []
for k in [5, 10, 15]:
    rmse, mae, recall_k, predicted_ratings = nmf_model_evaluation(pv_sparse_with_new_user, k=k)
    results.append([k, rmse, mae, recall_k])

# Convert results to a DataFrame and display in table format
results_df = pd.DataFrame(results, columns=["Top K", "RMSE", "MAE", "Recall@K"])
print("\nEvaluation Metrics for Different Recommendation Sizes:")
print(results_df.to_string(index=False))